# GoPro Photos to Crop KMZ

<a target="_blank" href="https://colab.research.google.com/github/nasaharvest/street2sat/blob/main/notebooks/GoPro2CropKMZ.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Author**: Ivan Zvonkov

**Last Modified**: Jul 8, 2024

**Description**: Converts GoPro photos to crop type points. Specifically the notebook:

1. Downloads GoPro photos from street2sat bucket.
2. Create a dataframe from photos.
3. Extract dates and coordinates.
4. Classify as crop or not crop.
5. Segment crops in crop photos.
6. Filter out low confidence predictions.
7. Move coordinate to field
8. Get Admin Zones for each point
9. Create KMZ file
10. Save the notebook



## Important Prerequisite

Before running any cell in the notebook,
1. Click the drop down triangle on the top right hand side and select "Change Runtime Type".
2. Click the T4 radio button under Hardware Accelerator and click save.

This will allow the CropSegmentation model to run much faster.

In [ ]:
# Required packages
!pip install exifread utm simplekml earthengine-api geemap -q

## 1. Download GoPro photos

In [ ]:
# Login to Google Cloud, once you run this cell click on the space after "browser:" to enter the code
!gcloud auth login
# Your current project will be [None] and that is okay.

In [ ]:
from pathlib import Path

GCLOUD_BUCKET = "street2sat-uploaded"

# Specify the folder name and download the images
GCLOUD_FOLDER = "KENYA_v2/2022_07_13_Nakuru_2"

GCLOUD_PATH = f"gs://{GCLOUD_BUCKET}/{GCLOUD_FOLDER}/"
PREFIX = GCLOUD_FOLDER.replace("/", "_")
Path(PREFIX).mkdir(exist_ok=True)
print(f"Ready to download images from {GCLOUD_PATH} to {PREFIX}")

In [ ]:
# Check amount of photos
!gsutil du $GCLOUD_PATH | wc -l

In [ ]:
# 20 mins for 10k images
!gsutil -m cp -r $GCLOUD_PATH $PREFIX

## 2. Create dataframe from available photos

In [ ]:
from datetime import datetime
from shapely.geometry import Point
from tqdm import tqdm

import exifread
import geopandas as gpd
import pandas as pd

tqdm.pandas()

In [ ]:
image_folder = Path("/content/KENYA_v2_2022_07_13_Nakuru_2/2022_07_13_Nakuru_2")
if (not image_folder.exists()):
    print("STOP: Update image_folder to match the folder of images you downloaded")
else:
    gopro_photo_paths = list(image_folder.glob("*.JPG")) # May need to switch to .jpg
    df = pd.DataFrame({"paths": gopro_photo_paths})

In [ ]:
df

## 3. Extract date and coordinate from each available photo

In [ ]:
def extract_date_lat_lon(img_path):
    img_bytes = open(img_path, "rb")
    tags = exifread.process_file(img_bytes)
    if tags == {}:
        print("No exiftags found")
        return None, None, None

    # Extract date
    image_datetime = str(tags.get("Image DateTime"))

    # Convert to Python datetime object
    dt = datetime.strptime(image_datetime, "%Y:%m:%d %H:%M:%S")

    def convert_to_degrees(coord):
        """ Convert the GPS coordinates stored in the EXIF to degress in float format"""
        d = float(coord.values[0].num) / float(coord.values[0].den)
        m = float(coord.values[1].num) / float(coord.values[1].den)
        s = float(coord.values[2].num) / float(coord.values[2].den)
        return d + (m / 60.0) + (s / 3600.0)

    lat = convert_to_degrees(tags.get("GPS GPSLatitude"))
    lon = convert_to_degrees(tags.get("GPS GPSLongitude"))

    if tags.get("GPS GPSLatitudeRef").values[0] != "N":
        lat = 0 - lat
    if tags.get("GPS GPSLongitudeRef").values[0] != "E":
        lon = 0 - lon

    return dt, lat, lon

# Extract date and lat lon from each image
df[["date", "lat", "lon"]] = df["paths"].progress_apply(extract_date_lat_lon).to_list()

In [ ]:
df

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=[Point(xy) for xy in zip(df["lon"], df["lat"])])

In [ ]:
# TODO better point plotting

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
region = world[world["name"] == "Kenya"]
ax = region.plot(facecolor="lightgray", figsize=(15, 15));

gdf.plot(
    ax=ax,
    marker='o',
    categorical=True,
    markersize=1,
    #column=DATASET,
    legend=True,
    legend_kwds={'loc': 'lower left'});

## 4. Classify as crop or not crop

In [ ]:
# Download CropNop model weights
!gsutil cp gs://street2sat-models/cropnop_v1.torchscript.pt .

In [ ]:
import cv2
import matplotlib.pyplot as plt
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load CropNop model
cropnop_model = torch.jit.load("/content/cropnop_v1.torchscript.pt")

def is_crop_or_not(img_path):

    # Preprocess image
    img = plt.imread(img_path)
    img = cv2.resize(img, (300, 300)) / 255
    img = img.transpose(2, 0, 1).astype("float32")
    img_tensor = torch.from_numpy(img).float().to(device)

    # Make crop or not prediction
    output = cropnop_model(img_tensor.unsqueeze(0))
    is_crop = (output <= 0).item()
    return is_crop

In [ ]:
# CropNop Example
img_path = df["paths"].loc[8]
is_crop_prediction = is_crop_or_not(img_path)

plt.imshow(plt.imread(img_path))
print(f"CropNop Model Prediction: {'Crop' if is_crop_prediction else 'Not crop'}")

In [ ]:
# 20 mins for 10k images
df["is_crop"] = df["paths"].progress_apply(is_crop_or_not)

In [ ]:
df["is_crop"].value_counts()

## 5. Segment crops

In [ ]:
# Download CropSeg model weights
!gsutil cp gs://street2sat-models/cropseg_v1.torchscript.pt .

In [ ]:
from skimage.io import imread
from skimage.transform import resize

import numpy as np
import os

os.environ["LRU_CACHE_CAPACITY"] = "1"

# Load CropSeg model
cropseg_model = torch.jit.load("/content/cropseg_v1.torchscript.pt")

CLASSES = [
    "background",
    "banana",
    "maize",
    "rice",
    "soybean",
    "sugarcane",
    "sunflower",
    "tobacco",
    "wheat",
]

def segment_crops(img_path):
    img = imread(img_path)
    img = resize(img, (800, 800))
    img = img.astype(float)
    img = (
        255 * (img - np.min(img[:])) / (np.max(img[:]) - np.min(img[:]) + 0.1)
    ).astype(float)
    img = (img + 0.5) / 256
    gamma = -1 / np.nanmean(np.log(img))
    img = img ** (gamma)
    img_transposed = img.transpose(2, 0, 1).astype("float32")
    img_tensor = torch.from_numpy(img_transposed).unsqueeze(0).to(device)
    return img, cropseg_model(img_tensor)[0].cpu().detach().numpy()


def segment_crops_w_proportions(img_path):
    _, output = segment_crops(img_path)
    image_size = output.shape[1] * output.shape[2]
    segmentation_proportions = {
        crop:  round(output[i].sum() / image_size, 4) for i, crop in enumerate(CLASSES)
    }
    return segmentation_proportions

In [ ]:
df_crops = df[df['is_crop']].copy()

In [ ]:
# Example CropSeg Predictions
img_path = df_crops["paths"].iloc[0]
img, output = segment_crops(img_path)
props = segment_crops_w_proportions(img_path)

fig, axes = plt.subplots(1, 2, figsize=(12, 12))
axes[0].imshow(img)
axes[1].imshow(output.argmax(axis=0), cmap='tab10', vmin=0, vmax=len(CLASSES))
for crop, prop in sorted(props.items(), key=lambda item: item[1], reverse=True):
    if crop != "background" and prop > 0:
        print(f"{crop}: {prop}")

In [ ]:
# ~10 mins for 500 images, 1hr for 2k images
# TODO can probably make predictions faster through batches
df_crops["segmentation_proportions"] = df_crops["paths"].progress_apply(segment_crops_w_proportions)

In [ ]:
proportion_columns = pd.json_normalize(df_crops["segmentation_proportions"]).set_index(df_crops.index)
df_crop_prop = pd.concat([df_crops, proportion_columns], axis=1)

In [ ]:
crops = list(proportion_columns.columns[1:][1:])
if "dominant_crop" not in df_crop_prop.columns:
    df_crop_prop["dominant_crop"] = df_crop_prop[crops].apply(lambda x: max(dict(x), key=dict(x).get), axis=1)

## 6. Filter out low confidence predictions

In [ ]:
print(f"Before background filter: {len(df_crop_prop)}")

# Only points with less than 95% background kept
bg_threshold = 0.95
df_crops_filtered = df_crop_prop[df_crop_prop["background"] < bg_threshold ].copy()
print(f"After background filter: {len(df_crops_filtered)}")


## 7. Move coordinate to field

In [ ]:
import utm
from datetime import timedelta
import math

# TODO add visual example

In [ ]:
# Copied and pasted from field_coord_distance_offset.ipynb

floor10 = lambda x: x//10 * 10
to_pixel_centroid = lambda coord: (floor10(coord[0]) + 5, floor10(coord[1]) + 5)

def generate_offset_point_wgs84(coord0, coord1, is_right_hand_drive=True, meters=20):
    utm_coord0 = utm.from_latlon(coord0[0], coord0[1])
    utm_coord1 = utm.from_latlon(coord1[0], coord1[1])

    for i, zone_type in [(2, "number"), (3, "letter")]:
        if utm_coord1[i] != utm_coord0[i]:
            print(utm_coord0)
            print(utm_coord1)
            raise ValueError(f"UTM Zone {zone_type} mismatch: {utm_coord0[i]} and {utm_coord1[i]}")


    delta_east = utm_coord1[0] - utm_coord0[0]
    delta_north = utm_coord1[1] - utm_coord0[1]

    # Offset for meters change in offset point distance
    x_offset = np.abs(meters * math.cos(math.atan(delta_east / delta_north)))

    # Direction of offset
    x_direction = np.sign(delta_north) if is_right_hand_drive else -np.sign(delta_north)
    x_offset *= x_direction

    orthogonal_slope = -delta_east / delta_north
    orthogonal_b = utm_coord1[1] - (orthogonal_slope * utm_coord1[0])
    orthogonal_y = lambda x: orthogonal_slope*x + orthogonal_b

    field_point_x = utm_coord1[0] + x_offset
    field_point_y = orthogonal_y(field_point_x)

    field_latlon = utm.to_latlon(field_point_x, field_point_y, utm_coord1[2], utm_coord1[3])

    pixel_centroid_x, pixel_centroid_y  = to_pixel_centroid((field_point_x, field_point_y))
    pixel_centroid_field_latlon = utm.to_latlon(pixel_centroid_x, pixel_centroid_y, utm_coord1[2], utm_coord1[3])

    return field_latlon, pixel_centroid_field_latlon, (delta_east, delta_north)

def road_pixel_centroid(coord):
    utm_coord = utm.from_latlon(coord[0], coord[1])
    utm_pixel_centroid = to_pixel_centroid(utm_coord)
    return utm.to_latlon(*utm_pixel_centroid, utm_coord[2], utm_coord[3])

In [ ]:
field_points = []
is_right_hand_drive = False

for i in tqdm(range(0, len(df_crops_filtered))):

    # Get road coordinate
    current_record = df_crops_filtered.iloc[i]
    road_coord = current_record["lat"], current_record["lon"]
    road_10m_centroid = road_pixel_centroid(road_coord)

    # Get prior coordinate
    time1 = current_record["date"]
    before_time_interval = time1 - timedelta(seconds=10)
    time_filter = (df["date"] < str(time1)) & (df["date"] > str(before_time_interval))
    prior_records = df[time_filter].sort_values(by=['date'])
    if len(prior_records) == 0:
        print(f"No prior records found for {i}")
        continue

    prior_record = prior_records.iloc[-1]
    prior_coord = prior_record["lat"], prior_record["lon"]

    # Get direction and field offset
    try:
        output = generate_offset_point_wgs84(prior_coord, road_coord, is_right_hand_drive)
        offset_field_coord, offset_field_pixel_centroid, driving_direction = output

        field_points.append({
            "road_pixel_centroid": road_10m_centroid,
            "is_right_hand_drive": is_right_hand_drive,
            "driving_easting": driving_direction[0],
            "driving_northing": driving_direction[1],
            "offset_field_coord": offset_field_coord,
            "offset_field_pixel_centroid": offset_field_pixel_centroid,
            "time_computed": datetime.now(),
            **df_crops_filtered.iloc[i],
        })
    except Exception as e:
        print(f"Index: {i}, Exception: {e}")


In [ ]:
df_crop_type_w_duplicates = pd.DataFrame(field_points)
print(f"Points BEFORE deduplicating: {len(df_crop_type_w_duplicates)}")

In [ ]:
# Deduplication by selecting max dominant crop
def get_max_dominant_crop(group):
  max_idx = group.apply(lambda x: x[x['dominant_crop']], axis=1).idxmax()
  return group.loc[max_idx]

# create new DataFrame with no duplicate - using groupby and applying get_max_dominant_crop on each group
df_crop_type = df_crop_type_w_duplicates.groupby('offset_field_pixel_centroid').apply(get_max_dominant_crop).reset_index(drop=True)
print(f"Points AFTER deduplicating: {len(df_crop_type)}")

## 8. Get Admin Zones for each point

In [ ]:
# Obtain Admin Boundaries from GADM: https://gadm.org/data.html
gdf_gadm2 = gpd.read_file("https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_KEN_2.json")

In [ ]:
geometry = [Point(xy) for xy in zip(df_crop_type["lon"], df_crop_type["lat"])]
gdf_points = gpd.GeoDataFrame(df_crop_type, geometry=geometry, crs="EPSG:4326")
gdf_points_gadm2 = gpd.sjoin(gdf_points, gdf_gadm2, how='left', op='within')

df_crop_type["GADM1"] = gdf_points_gadm2["NAME_1"]
df_crop_type["GADM2"] = gdf_points_gadm2["NAME_2"]

In [ ]:
df_crop_type["GADM1"].value_counts()

In [ ]:
df_crop_type["GADM2"].value_counts()

## 9. Create KMZ file

In [ ]:
import simplekml

DATA_GADM1_ZONES =  "_".join(df_crop_type["GADM1"].unique())
if DATA_GADM1_ZONES not in PREFIX:
    PREFIX += f"_{DATA_GADM1_ZONES}"

DATA_YEARS = "_".join([str(year) for year in df_crop_type["date"].dt.year.unique()])
if DATA_YEARS not in PREFIX:
    PREFIX += f"_{DATA_YEARS}"

DATA_BG_THRESHOLD = f"bg{int(bg_threshold*100)}"
PREFIX += f"_{DATA_BG_THRESHOLD}"

# Change if necessary
PREFIX

In [ ]:
endpoint_prefix = GCLOUD_PATH.replace("gs://", "")


def create_description(record, image_path):
    image_name = Path(image_path).name
    endpoint = Path(endpoint_prefix + "/" + image_name)
    name = "-".join(endpoint.parts[2:-1]) + "-" + endpoint.stem
    return f"""
<img src='files/{image_name}' width='900px'/>
<br/>
<h2>{name}</h2>
<p>Capture Time: {record['date']}</p>
<a href='https://storage.cloud.google.com/{endpoint}'>
    https://storage.cloud.google.com/{endpoint}
</a>

<h2>Location</h2>
<p>GADM1: {record['GADM1']}</p>
<p>GADM2: {record['GADM2']}</p>
<p>Road Lat Lon: {record['lat']}, {record['lon']}</p>
<p>Field Lat Lon:  {record["offset_field_pixel_centroid"]}</p>


<h2>Driving Direction</h2>
<p>Northing: {record['driving_northing']}</p>
<p>Easting: {record['driving_easting']}</p>
<p>Is Right Hand Drive: {record['is_right_hand_drive']}</p>

<h2>CropSeg Model Prediction</h2>
<p>{record['segmentation_proportions']}</p>

"""

In [ ]:
# Create KMZ file for every 100 points (more points make the KMZ laggy)
num_records = len(df_crop_type)

for range_start in range(0, num_records, 100):
    if range_start + 100 < num_records:
        range_end = range_start + 100
    else:
        range_end = num_records

    kml_document_name = PREFIX + f"_{range_start}_{range_end}"

    kml = simplekml.Kml()
    kml.document.name = kml_document_name

    for _, record in tqdm(df_crop_type[range_start:range_end].iterrows()):
        latlon = record["offset_field_pixel_centroid"]
        image_path = record['paths']
        kml.newpoint(
            coords=[(latlon[1], latlon[0])],  # lon, lat optional height
            description=create_description(record, image_path),
            name=record["dominant_crop"],
            timestamp=record["date"]
        )
        kml.addfile(image_path)


    kml.savekmz(f"{kml_document_name}.kmz", format=False)

In [ ]:
# Upload KMZ files to my Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

!cp -r /content/*.kmz /content/gdrive/My\ Drive


The uploaded KMZ files can now be downloaded onto a computer with Google Earth Pro.

A Quality Assessment must be conducted following this protocol:

https://docs.google.com/document/d/1OCF2gpCQQbZP-y6xcTbKE2OzhkxMtyaJi8wiWi8jfzs/edit?usp=sharing

## 10. Save the notebook

1. Click File / Download / Download .ipynb
2. Rename this notebook to have the name: ` <PREFIX>.ipynb`


In [ ]:
print(f"Suggested notebook name: {PREFIX}.ipynb")